# Creamos una RAG "Retrieval-Augmented Generation" leemos un PDF alimentamos el modelo y lo analizamos con Llama-Index y OpenAI

### Instalamos paquetes y los importamos

In [62]:
!pip install llama-index -q
!pip install openai -q
!pip install pypdf -q
!pip install ipython -q
from llama_index.core import SimpleDirectoryReader
from llama_index.llms.openai import OpenAI
from llama_index.core import VectorStoreIndex
from IPython.display import Markdown, display
from llama_index.core import ServiceContext
from llama_index.core import Settings
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import StorageContext, load_index_from_storage

In [2]:
import openai
import os
openai.api_key="YourKeyApi" #Pones la clave de tu API de OpenAI

## Creamos una App RAG Basico

In [9]:
#Cargamos los documentos contenidos en la carpeta data "Pdf, xml, txt, etc"
documents = SimpleDirectoryReader("./data/").load_data()

In [20]:
#Confirmamos lectura del documento
print(type(documents[0]))
print(documents[0])

<class 'llama_index.core.schema.Document'>
Doc ID: 8ad077f2-6cb1-429e-bad4-a7c31ee8001e
Text: Ciencia de datos desde cero Principios básicos con Python 2.ª
Edición Joel Grus


In [21]:
index = VectorStoreIndex.from_documents(documents)  #Creamos los embeddings se usa el modelo ADA-002
query_engine = index.as_query_engine() #Creamos el motor y la ventana de contexto

#### Preguntamos al motor y este al modelo

In [27]:
response = query_engine.query(
    "Que es lo que mas destacas del libro"
)
print(str(response))

El libro destaca por su actualización y mejora en comparación con la primera edición. Se ha reescrito todo el código y los ejemplos utilizando Python 3.6, se han añadido nuevos temas como deep learning, estadísticas y procesamiento del lenguaje natural, y se ha hecho énfasis en escribir código limpio. Además, se han reemplazado algunos ejemplos por otros más realistas y se han corregido errores, reescrito explicaciones menos claras y actualizado algunos chistes.


* Respuesta clara, corta y concreta. Me gusta que confirma la existencia de chistes

### Personalizamos y le damos caracter al modelo

In [55]:
Settings.llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1) #seleccionamos el modelo y le damos un poco de azucar para que sea mas creativo
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

In [56]:
index = VectorStoreIndex.from_documents(
    documents, embed_model = Settings.embed_model
)

In [66]:
query_engine_OpenIA = index.as_query_engine(llm=Settings.llm) # Cambiamos el nombre de la variable Query, para diferenciarlo de la otra

In [58]:
response = query_engine_OpenIA.query(
    "Que es lo que mas destacas del libro"
)
print(str(response))

La meticulosa revisión y actualización del contenido, incluyendo la reescritura del código y ejemplos en Python 3.6, la incorporación de conjuntos de datos más realistas, la adición de material sobre temas actuales como deep learning, estadísticas y procesamiento del lenguaje natural, así como la corrección de errores y la mejora de las explicaciones, son aspectos destacados del libro.


### Creamos persistencia del Index en nuestro disco y no en memoria del VectoreStore

In [65]:
index.storage_context.persist() #Creamos la carpeta en la raiz, llamada Storege en el disco local y los guardamos

storage_context = StorageContext.from_defaults(persist_dir="./storage") #Leemos los indices almacenados en el Storage
index = load_index_from_storage(storage_context=storage_context) # Cargamos los indices y podremos ejecutar la Query de consulta

In [71]:
query_engine_Local = index.as_query_engine() # Cambiamos el nombre de la variable Query, para diferenciarlo de la otra

In [72]:
response = query_engine_Local.query(
    "El libro aborda temas de Hacking"
)
print(str(response))

Yes, the book covers topics related to hacking, specifically focusing on developing computer skills at a hacker level needed for working in data science.


* Curioso, nos respondio en ingles.

In [74]:
response = query_engine_Local.query(
    "Que conocimientos se deben tener, para leer y entender este libro "
)
print(str(response))

Para leer y entender este libro, se deben tener conocimientos en habilidades informáticas a nivel de hacker, matemáticas, estadística y experiencia relevante en ciencia de datos.


* La respuesta me parecio un poco.... fumada, por el termino Hacker. Pero por lo demas es clara y concisa.